In [1]:
import pyvisa
import time
import numpy as np
import pandas as pd
import sys

ExitMode=['exit','Exit','EXIT']
rm = pyvisa.ResourceManager()
resources=rm.list_resources()
instruments = np.array(resources)
print(resources)
devices = []
for resource in resources:
    try:
        instrument = rm.open_resource(resource)
#        print(f'{resource} is available')
        address = instrument.resource_name
        vendor = instrument.query("*IDN?").strip()
        devices.append([address, vendor, 'Available'])
        identity = instrument.query('*IDN?')
#        print(identity)
        instrument.close()
    except:
#        print(f'{resource} is not available')
        devices.append([resource,"","Not available"])

def classify_vendor(vendor):
    if 'MSO7034B' in vendor:
        return 'Oscilloscope'
    elif '34461' in vendor:
        return 'DMM'
    elif 'N3300' in vendor:
        return 'Load'
    elif 'AC6803' in vendor:
        return 'Input'
    else:
        return 'Other'
devices_df = pd.DataFrame(devices, columns=['Address', 'Vendor','Available?'])
devices_df['Function'] = devices_df['Vendor'].apply(classify_vendor)
dmm_mask = devices_df['Function'] == 'DMM'
other_mask = devices_df['Function'] != 'DMM'
dmm_addresses = (devices_df['Function'] == 'DMM') & (~devices_df['Address'].str.contains('MY57224092|MY57224113|MY57224258|MY57224696'))
                                
devices_df['Measure'] = np.where(dmm_mask & devices_df['Address'].str.contains('MY57224092'), 'Vin', 
                                np.where(dmm_mask & devices_df['Address'].str.contains('MY57224113'), 'Iin', 
                                         np.where(dmm_mask & devices_df['Address'].str.contains('MY57224258'), 'Vout',
                                                  np.where(dmm_mask & devices_df['Address'].str.contains('MY57224696'), 'Iout',np.nan))))

Oscil_Address= devices_df.loc[devices_df['Function'] == 'Oscilloscope', 'Address'].values[0]
# VinDMM_Address = devices_df.loc[(devices_df['Function'] == 'DMM')& (devices_df['Measure'] == 'Vin'), 'Address'].values[0]
# VoutDMM_Address = devices_df.loc[(devices_df['Function'] == 'DMM')& (devices_df['Measure'] == 'Vout'), 'Address'].values[0]
# Load_Address = devices_df.loc[devices_df['Function'] == 'Load', 'Address'].values[0]
# IinDMM_Address = devices_df.loc[(devices_df['Function'] == 'DMM')& (devices_df['Measure'] == 'Iin'), 'Address'].values[0]
# IoutDMM_Address = devices_df.loc[(devices_df['Function'] == 'DMM')& (devices_df['Measure'] == 'Iout'), 'Address'].values[0]
Vin_Address = devices_df.loc[devices_df['Function'] == 'Input', 'Address'].values[0]


# OutLoad = rm.open_resource(Load_Address)
Vin = rm.open_resource(Vin_Address)
Oscil=rm.open_resource(Oscil_Address)
# IinDMM = rm.open_resource(IinDMM_Address)
# IoutDMM = rm.open_resource(IoutDMM_Address)
# VinDMM = rm.open_resource(VinDMM_Address)
# VoutDMM = rm.open_resource(VoutDMM_Address)



devices_df

('GPIB0::32::1::INSTR', 'GPIB0::5::INSTR', 'TCPIP0::192.168.0.118::inst0::INSTR', 'USB0::0x0957::0x173D::MY50340340::0::INSTR', 'USB0::0x0957::0xAC07::JPWC002359::0::INSTR')


,Address,Vendor,Available?,Function,Measure
0,GPIB0::32::1::INSTR,,Not available,Other,nan
1,GPIB0::5::INSTR,,Not available,Other,nan
2,TCPIP0::192.168.0.118::inst0::INSTR,"AGILENT TECHNOLOGIES,MSO7034B,MY50340340,06.15...",Available,Oscilloscope,nan
3,USB0::0x0957::0x173D::MY50340340::0::INSTR,"AGILENT TECHNOLOGIES,MSO7034B,MY50340340,06.15...",Available,Oscilloscope,nan
4,USB0::0x0957::0xAC07::JPWC002359::0::INSTR,"Agilent,AC6803A,JPWC002359,A.01.02.0088",Available,Input,nan


In [ ]:
###############--CONFIGURE LOAD--############################

OutLoad.write("INP ON")
OutLoad.write("CHANnel 2") # Load channel we are using
OutLoad.write("FUNC CURR")
OutLoad.write("CURR 1")

In [2]:


###############--CONFIGURE INPUT--############################
print('Configuring Input')

Vin.write("*RST") #Reset to default settings
Vin.write("OUTP:COUPLing DC")
Vin.write("SOUR:VOLT:OFFS 80")
# Vin.write("SOUR:VOLTage:RANGE AUTO")
Vin.write("SOUR:CURRent:OFFSET 8")


if input('Testing in Boost mode at 1 A. Press ENTER to continue or EXIT to cancel') in ExitMode :
    sys.exit()
Vin.write("OUTP 1") #Enable Vin source output AFTER configuring load (safety purposes)
input('START TURN ON SEQUENCE ON FPGA (switches). Verify it works. Press ENTER TO CONTINUE if all OK')

vinmeas = Vin.query_ascii_values("MEAS:VOLT?")
iinmeas = Vin.query_ascii_values("MEAS:CURR?")
print("Voltage set is {}. Current consumption is set to {}.".format(vinmeas[0],iinmeas[0]))
# GoOn=input(' Should we STOP MEASURING?. Press "n"')
# while GoOn!='n':
#     GoOn=input(' Should we STOP MEASURING?. Press "n"')
# Vin.write("OUTP 0")

 
        




Configuring Input
Voltage set is 80.0036. Current consumption is set to -0.0175903.


## Save oscilloscope signal

In [11]:
Oscil.write(":SAVE:FILEname 'ZVSbck_5A12'")
print("Saving file")
NameSave = Oscil.query(":SAVE:FILEname?")
print('Saving file as {}'.format(NameSave))


Oscil.write(":SAVE:PWD '/usb0/vac2/'")
print(Oscil.query(":SAVE:PWD?"))

Oscil.write(":SAVE:IMAGe:FORMat PNG")
Oscil.write(":SAVE:IMAGe:INKSaver OFF")
Oscil.write(":SAVE:IMAGe:PALEtte COLOr")
# while not bool(Oscil.query("*opc?")): #wait until saving finished
#     pass
# time.sleep(3)
Oscil.write(":SAVE:IMAGe:STARt {}".format(NameSave))
# Oscil.write(":SAVE:WAVeform:FORMat CSV")
# Oscil.write(":SAVE:WAVeform:LENGth 1000")
# time.sleep(3)
# Oscil.write(":SAVE:WAVeform:STARt {}".format(NameSave))

Saving file
Saving file as "ZVSbck_5A12"

"/usb0/vac2/"



34

ValueError: only recognize index or columns for orient

['GPIB0::5::INSTR']

## AC Source Input

Voltage was 29.9498. Current was -0.0133115.


Resource: 'GPIB0::5::INSTR' is
Agilent Technologies,N3300A,0,A.00.08


Resource: 'USB0::0x0957::0xAC07::JPWC002359::0::INSTR' is
Agilent,AC6803A,JPWC002359,A.01.02.0088




VisaIOError: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.

## Load

In [2]:
Load=['GPIB0::5::INSTR']
resource_manager = pyvisa.ResourceManager()

OutLoad = resource_manager.open_resource(Load[0])

OutLoad.write("*RST")
OutLoad.write("INP ON")
OutLoad.write("CHANnel 2")
OutLoad.write("FUNC CURR")
OutLoad.write("CURR 1.5")
OutLoad.close()

## DMM

In [8]:
import pyvisa
import time
DMM=['USB0::0x2A8D::0x1301::MY57224113::0::INSTR']
resource_manager = pyvisa.ResourceManager()
VinDMM = resource_manager.open_resource(DMM[0])
VinDMM.write('DISP:TEXT "Iin Meas"')
VinDMM.write("*RST")
VinDMM.write("*CLS")
VinDMM.write("CONF:CURR:DC AUTO")
VinDMM.write("CURR:DC:TERMinals 10")
# print(VinDMM.query("SYST:ERROR?"))
VinDMM.write("TRIG:DEL:AUTO ON")
VinDMM.query("*OPC?")
Vrcv=VinDMM.query("READ?")

# EQUIVALENT TO 'READ?': print(VinDMM.query("MEAS:VOLT:DC?"))
print(Vrcv)

VinDMM.write('DISP:TEXT:CLE')
print(VinDMM.query("SYST:ERROR?"))

#VinDMM.write("INIT")
#VinDMM.write("SENS: DATA2")
#VinDMM.write("")
#VinDMM.write("")
VinDMM.close()



+3.44497805E+00

+0,"No error"



## Oscilloscope

In [41]:
Oscil=['USB0::0x0957::0x173D::MY50340340::0::INSTR']
Measurement=['test1']
resource_manager = pyvisa.ResourceManager()
Oscil = resource_manager.open_resource(Oscil[0])
Oscil.encoding = 'ascii'


DMM_sequence=['Vinmeas','Iinmeas','Voutmeas','Ioutmeas','Pin','Pout','Eff']
Oscil_sequence=['Vinput','Vinput AC','Voutput','Voutput AC','Iload','Iload AC','Iinput','Iinput AC','Ioutput','Ioutput AC','Vinput sens','Iinput sens','Voutput sens','Ioutput sens','Iload sens']
# Test_sequence = DMM_sequence+Oscil_sequence
Test_sequence=['Vinput','Vinput AC','Voutput','Voutput AC','Iload','Iload AC','Iinput','Iinput AC','Ioutput','Ioutput AC','Vinput sens','Iinput sens','Voutput sens','Ioutput sens','Iload sens']
LoadCurr=['1','5'] # A
ConvMode=['Buck','Boost']
ConvModeVoltageDict={'Buck':'150','Boost':'80'}
files=['CSV', 'PNG']
# SAME AS THE OTHER ONE BUT CHATGPT WROTE IT ONE BY ONE. KEPT IN CASE MORE CHANGES NEEDE ETO BE MADE
# ChanConfLevel = {
#     'Vinput': ['50','500mV','2','2'],
#     'Vinput AC': ['100mV','500mV','2','2'],
#     'Voutput': ['50','500mV','2','2'],
#     'Voutput AC': ['100mV','500mV','2','2'],
#     'Iload': ['5','500mV','2','2'],
#     'Iload AC': ['5','500mV','2','500mA'],
#     'Iinput': ['5','500mV','2','2'],
#     'Iinput AC': ['5','500mV','2','500mA'],
#     'Ioutput': ['5','500mV','2','2'],
#     'Ioutput AC': ['5','500mV','2','500mA'],
#     'Vinput sens': ['5','500mV','2','2'],
#     'Iinput sens': ['5','500mV','2','2'],
#     'Voutput sens': ['5','500mV','2','2'],
#     'Ioutput sens': ['5','500mV','2','2'],
#     'Iload sens': ['50mV','500mV','2','2']
# }

ChanConfLevel = {}
for item in Test_sequence:
    if item in ['Iload', 'Iinput', 'Ioutput','Vinput sens','Iinput sens','Voutput sens','Ioutput sens','Iload sens']:
        ChanConfLevel[item] = ['5', '500mV', '2', '2']
    elif item in ['Vinput', 'Voutput']:
        ChanConfLevel[item] = ['50', '500mV', '2', '2']
    elif item in ['Vinput AC', 'Voutput AC']:
        ChanConfLevel[item] = ['100mV', '500mV', '2', '2']
    elif item in ['Iinput AC', 'Ioutput AC']:
        ChanConfLevel[item] = ['5', '500mV', '2', '500mA']
    elif item == 'Iload sens':
        ChanConfLevel[item] = ['50mV', '500mV', '2', '2']
    else:
        ChanConfLevel[item] = ['5', '500mV', '2', '2']

ACDCConf= {}
for item in Test_sequence:
    if item in ['Vinput AC', 'Voutput AC']:
        ACDCConf[item]=['AC','AC','DC','DC']
    elif item in ['Iinput AC', 'Ioutput AC','Iload AC']:
        ACDCConf[item]=['DC','DC','AC','AC']
    elif item in ['Iload sens']:
        ACDCConf[item]=['AC','DC','AC','AC']
    
    else:
        ACDCConf[item]=['DC','DC','DC','DC']

TrigConf={}
for item in Test_sequence:
    for val in LoadCurr:
        if item in ['Iload AC']:
            TrigConf[val,item]='0.5'
        else:
            TrigConf[val,item]=val



multiindex = pd.MultiIndex.from_product([ConvMode, LoadCurr], names=["ConvMode", "LoadCurr"])
df = pd.DataFrame(index=multiindex, columns=pd.MultiIndex.from_product([Oscil_sequence, files], names=["Measurement", "Files"]))

DMM_df=pd.DataFrame(index=multiindex, columns=pd.MultiIndex.from_product([DMM_sequence], names=["Measurement"]))

###############--CONFIGURE OSCILLOSCOPE--####################
Oscil.write('*RST')
Oscil.write("*CLS")
ChanNum=4
Oscil.write("TIMebase:MODE MAIN")
Oscil.write("TIMebase:POSition 0")
Oscil.write("TIMebase:RANGe 1 ms")
Oscil.write("TIMebase:REFerence CENTer")
Oscil.write("TRIGger:SWEep AUTO")
Oscil.write("TRIGger:NREJect 1")
Oscil.write("TRIGger:HFReject 1")
Oscil.write("TRIGger:MODE Edge")#Selects Trigger mode
Oscil.write("TRIGger:EDGE:SOURce CHAN3")#Trigger source is channel 3 
Oscil.write("TRIGger:EDGE:LEVel 3") #Trigger level at 2 A
for i in range(ChanNum):
    n=i+1
    Oscil.write(":CHAN{}:DISP ON".format(n))
    Oscil.write(":CHAN{}:BWLimit ON".format(n))
    Oscil.write(":CHAN3:SCALe 2") #config level
    Oscil.write(":CHAN3:OFFSet 3")
    # Oscil.write(":TRIG:EDGE:LEVEl {}".format(TrigConf['1','Iload AC']))#config trigger
    # Oscil.write(":CHAN{}:COUPling {}".format(n,ACDCConf['Iload'][i]))#config coupling


            # Oscil.write(":SAVE:FILEname '{}'".format(element))
            # print("Saving file {}".format(element))
            # NameSave = Oscil.query(":SAVE:FILEname?")
            # print('Saving file as {NameSave}'.format(NameSave))
            
            # Oscil.write(":SAVE:PWD '/usb0/prueba/'")
            # print(Oscil.query(":SAVE:PWD?"))
            
            # Oscil.write(":SAVE:IMAGe:FORMat PNG")
            # Oscil.write(":SAVE:IMAGe:INKSaver OFF")
            # Oscil.write(":SAVE:IMAGe:PALEtte COLOr")
            # Oscil.write(":SAVE:IMAGe:STARt {}".format(NameSave))
            # df.loc[(converter,load_current),(element,'PNG')]=NameSave+'.png'

            # Oscil.write(":SAVE:WAVeform:FORMat CSV")
            # Oscil.write(":SAVE:WAVeform:LENGth 1000")
            # Oscil.write(":SAVE:WAVeform:STARt {}".format(NameSave))
            # df.loc[(converter,load_current),(element,'CSV')]=NameSave+'.csv'




# for meas in Measurement:

#     Oscil.write('*RST')
#     Oscil.write("*CLS")
#     ChanNum=4
#     Oscil.write("TIMebase:MODE MAIN")
#     Oscil.write("TIMebase:POSition 0")
#     Oscil.write("TIMebase:RANGe 1 ms")
#     Oscil.write("TIMebase:REFerence CENTer")
#     Oscil.write("TRIGger:SWEep AUTO")
#     Oscil.write("TRIGger:NREJect 1")
#     Oscil.write("TRIGger:HFReject 1")
#     Oscil.write("TRIGger:MODE Edge")#Selects Trigger mode
#     Oscil.write("TRIGger:EDGE:SOURce CHAN3")#Trigger source is channel 3 
#     Oscil.write("TRIGger:EDGE:LEVel 2") #Trigger level at 2 A
#     CHsep=pd.DataFrame()
#     for i in range(ChanNum):
#         n=i+1
#         Oscil.write(":CHAN{}:DISP ON".format(n))
#         Oscil.write(":CHAN{}:BWLimit ON".format(n))
#         Oscil.write("WAVEForm:Source CHAN{}".format(n))
#         Oscil.write("WAVEForm:points max")
#         Oscil.write("WAVEForm:format ASCII")
#         c1=Oscil.query("WAVEform:DATA?")

#         c1=c1.replace('#800012999','')
#         c1=c1.split(',')
#         c1.append(meas for i in len(c1))

#         CHsep['CH{}'.format(n)]=[float(point) for point in c1]
#         # print('Esto es CH{}:   {}'.format(n,c1))
#     # create a list of tuples for each channel
        
#     # convert the list of tuples to a DataFrame
# # df = pd.DataFrame(CHsep, columns=['Measurement', 'Channel1', 'Channel2', 'Channel3', 'Channel4'])
#     # set the multi-index using the elements of Test_sequence
# # df = df.set_index('Measurement')
#     # Oscil.write(":SAVE:FILEname '{}'".format(meas))
#     # print(":SAVE:FILEname {}".format(meas))
#     # NameSave =Oscil.query(":SAVE:FILEname?")
#     # print(NameSave)
#     # Oscil.write(":SAVE:PWD '/usb0/prueba/'")
#     # print(Oscil.query(":SAVE:PWD?"))

#     # Oscil.write(":SAVE:IMAGe:FORMat PNG")
#     # Oscil.write(":SAVE:IMAGe:INKSaver OFF")
#     # Oscil.write(":SAVE:IMAGe:PALEtte COLOr")

#     # Oscil.write(":SAVE:IMAGe:STARt {}".format(NameSave))
#     # Oscil.write(":SAVE:WAVeform:FORMat CSV")
#     # Oscil.write(":SAVE:WAVeform:LENGth 1000")
#     # Oscil.write(":SAVE:WAVeform:STARt {}".format(NameSave))




df

Measurement       Vinput      Vinput AC      Voutput      Voutput AC       \
Files                CSV  PNG       CSV  PNG     CSV  PNG        CSV  PNG   
ConvMode LoadCurr                                                           
Buck     1           NaN  NaN       NaN  NaN     NaN  NaN        NaN  NaN   
         5           NaN  NaN       NaN  NaN     NaN  NaN        NaN  NaN   
Boost    1           NaN  NaN       NaN  NaN     NaN  NaN        NaN  NaN   
         5           NaN  NaN       NaN  NaN     NaN  NaN        NaN  NaN   

Measurement       Iload       ... Vinput sens      Iinput sens       \
Files               CSV  PNG  ...         CSV  PNG         CSV  PNG   
ConvMode LoadCurr             ...                                     
Buck     1          NaN  NaN  ...         NaN  NaN         NaN  NaN   
         5          NaN  NaN  ...         NaN  NaN         NaN  NaN   
Boost    1          NaN  NaN  ...         NaN  NaN         NaN  NaN   
         5          NaN  NaN  ...         NaN  NaN         NaN  NaN   

Measurement       Voutput sens      Ioutput sens      Iload sens       
Files                      CSV  PNG          CSV  PNG        CSV  PNG  
ConvMode LoadCurr                                                      
Buck     1                 NaN  NaN          NaN  NaN        NaN  NaN  
         5                 NaN  NaN          NaN  NaN        NaN  NaN  
Boost    1                 NaN  NaN          NaN  NaN        NaN  NaN  
         5                 NaN  NaN          NaN  NaN        NaN  NaN  

[4 rows x 30 columns]

In [30]:
Oscil.query(":SAVE:FILEname?")
bool(Oscil.query("*opc?"))

True

In [16]:
import numpy as np
1/(np.sqrt(2*170e-9*170e-12)*2*np.pi)

20934192.453446385

In [17]:
1/((1.5e6*2*np.pi)**2*170e-6)

6.622299584466522e-11